# Control de pérdidas

### Mensaje _(Contenido en 'example.txt')_:

'123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051  5251234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950  5152512345678910111213141516171819202122232425262728293031323334353637383940414243444546474849  
5051525748495051525'

El cual toma 300 bytes, el cual es el tamaño máximo que admite el perfil de 1-byte Header. Esto da lugar a 27 fragmentos de 11 bytes cada uno, y a un último fragmento de 3 bytes.

Cada ventana alojará hasta 7 fragmentos (tiles). Por lo que será necesario un total de 4 ventanas. Se representará cada ventana por una linea de numeros, cuyo largo será de 7 a excepción de la última, que puede ser menor. Cada número indicará la cantidad de retransmisiones necesarias para llevar a cabo la comunicación, debido a pérdidas inducidas. En este caso, un ejemplo sería:

0000000  
0000000  
0000000  
0000000

Donde el mensaje se enviará sin contemplar pérdidas




### Observaciones
1.- Dado que el protocolo SCHC establece que se deben actualizar el número de Attemps en el sender; tras enviar un All-1 o un ACK REQ. Esto podría inducir a sobreestimar los reintentos.

2.- El protocolo SCHC también establece que un ACK REQ se envía luego de un All-1 o un reintento de tiles. (No necesariamente después de un All-0) 

## Caso 1: 

0010000    
0000000  
0000000  
0000000  

* Se espera que se pierda sólo el tercer fragmento de la primera ventana
* Este será retransmitido luego de que se envíe el All-0 y se reciba el ACK correspondiente antes de que expire el timer.




## Caso 2:

0010000  
0000001  
0100000  
0000010

* Se espera que se pierda un tile de cada ventana.

* Al enviar el All-0 de la ventana 00 se espera por el ACK. Luego de recibir el ACK se envía la tercera tile. Se prosigue con la ventana siguiente.

* Al enviar el All-0 de la ventana 01 se espera por el ACK. Como el ALL-0 se pierde, no llega ACK y el timer expira. Se prosigue con la ventana siguiente.

* Al enviar el All-0 de la ventana 02 se espera por el ACK. El ACK reporta pérdidas de la ventana 01. Se transmite exitosamente el ALL-0 de dicha ventana.

* Se continúa con la transmisión de la ventana 03. Una vez enviado el ALL-1 se espera por el ACK. El ACK reporta las pérdidas de la ventana 02. Se transmiten exitosamente dichas tiles.

* Se transmite nuevamente el ALL-1. Se espera por el ACK que reportará las pérdidas de la ventana 03. Una vez recibido se retransmite exitosamente el tile faltante.

* Ya finalizando, se envía el ALL-1 y se finaliza la transmisión.


## Caso 3:


0010000  
0000000  
0100002  
0000000  

* Se espera que se pierdan tiles en las ventanas 00 y 02

* Al enviar el All-0 de la ventana 00 se espera por el ACK. Luego de recibir el ACK se reenvía la tercera tile. Se prosigue con la ventana siguiente.

* Se transmite la ventana 01 sin complicaciones.

* Al enviar el All-0 de la ventana 02 se espera por el ACK. Luego, como el All-0 se perdió expira el timer. 

* Se transmite la ventana 03 sin complicaciones. El ACK recibido posteriormente informa de las pérdidas de la ventana 02

* Se transimiten los tiles faltantes de la ventana 02, sin embargo el ALL-0 se vuelve a perder. Se envía un All-1. 

* Luego del ALL-1 llega un ACK informando de la perdida del ALL-0. Se reenvía dicho tile.

* Se envía un último ALL-1 y la comunicación termina de forma exitosa




## Caso 4 (Comentar):

0000002  
0010000  
0000000  
0000000  

* Se espera que se pierdan tiles en las ventanas 00 y 01

* Al enviar el All-0 de la ventana 00 se espera por el ACK. Como el All-0 se perdió, el ACK no llega, luego expira el timer y se procede a enviar la siguiente ventana

* Al enviar el All-0 de la ventana 01 se espera por el ACK. Luego de recibir el ACK, se envía el ALL-0 de la ventana 0 (All-0).

* Como el tile nuevamente se pierde, el ACK no llega, por lo tanto expira el timer. La transmisión continúa desde la ventana 02.

* Al enviar el ALL-0 de la ventana 02, se reportan las pérdidas del ALL-0 de la ventana 0. Se reenvía el All-0.

* Luego, se continúa con la transmisión de la ventana 03, luego de enviar el ALL-0 se espera por un ACK

* El ACK reporta las pérdidas en la ventana 01. Se reenvía el tile faltante.

* Como está todo bien, no es necesario un ACK, y la comunicación continúa sin inconvenientes hasta finalizar.

## Caso 4.5 

0000003  
0001000  

* Se transmite la ventana 00, pero se pierde el ALL-0

* Al perderse el ALL-0, no existe ACK, por lo que el timer expira y se continúa con la transimisión de la segunda ventana (ventana 1).

* El ACK proveniente de la ventana 01 informa de las pérdidas de la ventana 00. Se envía el tile faltante, pero este se pierde, por lo tanto no llega ACK. 

* Al expirar el timer y haber mandado ya las dos ventanas, se envía el ALL-1. 

* Recibido el All-1, Se envía el ACK reportando pérdidas de la ventana 00. Se manda el All-0 pero el tile se pierde por segunda vez.

* Se reenvía el ALL-1. 

* Recibido el All-1, Se envía el ACK reportando pérdidas de la ventana 00. Se manda el All-0 pero el tile se pierde por tercera y última vez.

* Se reenvía el ALL-1 y la trnasmisión finaliza de manera exitosa


## Caso 5 *(Se pierde el All-0)*:

0000002  
0030000  
0000010  
0000000  

* Se transmite la ventana 00 pero se pierde el último tile. Por lo tanto se prosigue con la transmisión de la ventana 01

* Luego de transmitir la ventana 01 (con errores) se recibe un ACK informando de la falencia del ALL-0 en la ventana anterior.

* Se reenvía el tile pero se pierde nuevamente. Se continúa con la transmisión de la ventana 02.

* Se transmite la ventana 02 pero se extravía el séptimo tile. Luego de esto, se recibe un ACK indicando la falencia del último tile de la ventana 00

* Se reenvía por segunda vez el ALL-0 la ventana 0. Esta vez de manera exitosa

* Se transmite la ventana 03 sin complicaciones. Después del ALL-1 (attemps = 1) se recibirá un ACK indicando las pérdidas de la ventana 01.

* Al reenviarse la tercera tile de la ventana 01, se prosigue a enviar nuevamente el ALL-1 (attemps = 2)

* Se recibe un ACK informando la ausencia de la tercera tile en la ventana 01. Se reenvía dicha tile por segunda vez, y se pierde nuevamente. Se envía el ALL-1 (attemps = 3)

* Llega un ACK indicando la ausencia de tiles en la ventana 01. Se reenvía por tercera y última vez dicha tile. Se envía el ALL-1. (attemps = 4)

* Envido el ALL-1 se recibe un ACK el cual indica pérdidas en la ventana 02. Se reenvia el tile faltante sin complicaciones. Posteriormente se envía un All-1 (attemps = 5) y se finaliza la conexión.




## Caso 6 *(Se pierde el All-1)* :

0000000  
0000000  
0000000  
0000001   

* Se espera que se pierda el All-1

* Al finalizar la transimisión de la última ventana, el Timer del sender expira y se vuelve a enviar el All-1  

* Se recibe el ACK correspondiente y se termina la transmisión de manera exitosa.

## Caso 7

0000000  
0000000  
0000000  
0000011   

* Se espera que se pierdan la penúltima tile y el All-1

* Al finalizar la transimisión de la última ventana, el Timer del sender expira, por lo que se vuelve a enviar el All-1

* Al enviar el All-1 el server debe enviar el ACK. En el ACK viene la información de la penúltima tile faltante

* Se reenvía la penúltima tile

* Finaliza la transmisión de manera exitosa